In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.morphology

# Plotting tools
import bokeh
import bokeh_catplot
bokeh.io.output_notebook()

___

This notebook computes the maximum projection of confocal z-stack images. It then saves the images in a separate folder with or without a scalebar (define below). Currently it does not have the option to exclude channels or change the colors assigned (channel 1 will be red, channel 2 will be green, channel 3 will be blue), yet.  

Written by Laura Luebbert, 15th of May 2020.  

Modified on: / 

___

# Define the parameters

### Define the directory containing the tif files:

In [1]:
data_dir = ''

### Define interpixel distance and length unit:

In [ ]:
interpixel_distance = 0.3452670
length_units = "µm"

### Scale bar options:

In [ ]:
scale_bar = True

# Set scalebar color ("white" or "black")
scale_bar_color = "white"

# Define the desired scale bar size and width in length unit.
scale_bar_length = 50
scale_bar_width = 5

# Define position of scale bar:
# Distance from top (Distance in % of total length of image)
y_pos = 95
# Distance of rightmost end of scale bar from left end of image (Distance in % of total length of image)
x_pos = 95

### Saving options:

In [ ]:
# Directory where maximum projection image folder will be created.
saving_dir = data_dir

file_format = "tif"

### Filter options:

In [ ]:
# Despeckling using median filter (skimage.filters.rank.median)
despeckle = "Yes"
despeckle_parameter = 3

# Contrast is optimized using the skimage.exposure.equalize_adapthist function
optimize_contrast = "Yes"

___

# Load in the data

In [ ]:
# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, '*.tif')

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

___

# Compute maximum projections
Compute maximum pixel value of frames in each channel for each tif file, despeckle images, maximize contrast and then store them in array "max_ims":

### Compute max projection of each channel and merge channels

In [ ]:
max_ims = []

# Set median filter for despeckling
selem = skimage.morphology.square(despeckle_parameter)

for i, file in enumerate(im_list):
           
    # Read in each tif file using skimage
    image = skimage.io.imread(file)
        
    # For images containing more than one channel:           
    if len(image.shape) == 4:  
        # Create matrix of zeros with dimensions for channels with each x and y pixels
        max_channels = np.zeros((3, image.shape[2], image.shape[3]))

        # Matrix of zeros in the same size as our image (x-pixels, y-pixels)
        zeros = np.zeros((image.shape[2], image.shape[3]))

        for num, channel in enumerate(image): 
            # np.max with axis=0 returns the maximum of each column (each row equals to a frame); num = channel
            max_channel = image[num].max(axis=0)

            if despeckle == True:
                max_channel = skimage.filters.rank.median(max_channel, selem)   

            if optimize_contrast == True:                
                max_channel = skimage.exposure.equalize_adapthist(max_channel)

            max_channels[num] = max_channel
        
        max_ims.append(np.dstack((max_channels[0], max_channels[1], max_channels[2])))
        
    # For images containing only one channel:        
    elif len(image.shape) == 3:  
        # Create matrix of zeros with dimensions x and y pixels
        max_channels = np.zeros((image.shape[1], image.shape[2]))

        # Matrix of zeros in the same size as our image (x-pixels, y-pixels)
        zeros = np.zeros((image.shape[1], image.shape[2]))

        # np.max with axis=0 returns the maximum of each column (each row equals to a frame)
        max_channel = image.max(axis=0)

        if despeckle == "Yes":
            max_channel = skimage.filters.rank.median(max_channel, selem)   

        if optimize_contrast == "Yes":                
            max_channel = skimage.exposure.equalize_adapthist(max_channel)
        
        max_ims.append(np.dstack((max_channel, zeros, zeros)))
        
    else:
        raise TypeError("Image shape should be (channels, frames, x_pixels, y_pixels) or (frames, x_pixels, y_pixels)")

___

# Save merged maximum projections

#### Scale down images:

To save the images using skimage, we need to scale them down to 8 bit.

In [ ]:
# Scale down images to 8 bits
max_ims_8 = []

for i, file in enumerate(max_ims):
    # Linearly scale image down to 8-bit.
    image = (file / file.max()) * 255

    # Change list to array and change type to 8-bit array.
    image = np.array(image)
    image = image.astype(np.uint8)

    max_ims_8.append(image)

### Display the first merged maximum projection:

In [ ]:
skimage.io.imshow(max_ims_8[0])

#### Burn in scale bars:

Scale bar is burned into image by changing the pixel value to 1000 (white scale bar) or 0 (black scale bar) in scale bar area defined in "parameters" cell:

In [ ]:
if scale_bar == True:
  
    max_ims_8_with_scalebars = []

    scalebar = 1 / interpixel_distance * scale_bar_length
    scalebar_width = 1 / interpixel_distance * scale_bar_width

    if scale_bar_color == "white":
        for image in max_ims_8:
            y_value = int((image.shape[0]/100)*y_pos)
            x_value = int((image.shape[1]/100)*x_pos) 
            
            image[y_value : y_value + int(scalebar_width), x_value - int(scalebar) : x_value] = 255

            # Append to array.
            max_ims_8_with_scalebars.append(image)
            
    elif scale_bar_color == "black":
        for image in max_ims_8:
            y_pos = int((image.shape[0]/100)*y_pos)
            x_pos = int((image.shape[1]/100)*x_pos)
            
            image[y_pos : y_pos + int(scalebar_width), x_pos - int(scalebar) : x_pos] = 0

            # Append to new array.
            max_ims_8_with_scalebars.append(image)
            
else:
    max_ims_8_with_scalebars = max_ims_8

Display one image with scale bar:

In [ ]:
skimage.io.imshow(max_ims_8_with_scalebars[2])

### Create max projection folder:

Create folder in saving directory to save maximum projections to:

In [ ]:
path = ("{}/{}_max_projections").format(saving_dir, im_list[0].split("/")[-2])

os.mkdir(path)

#### Get image names:

Slice out image names:

In [ ]:
# Save the filename of the image in array as a first step to get the image title.
files = []

for filename in im_list:
    files.append(filename.split("/")[-1])

# Save image titles in array
imnames = []

for name in files:
    imnames.append(name.split(".")[0])

#### Save:

In [ ]:
# Save all images with the scale bar.
for i, image in enumerate(max_ims_8_with_scalebars):
    skimage.io.imsave(
        ("{}/{}_max.{}").format(path, imnames[i], file_format),
        max_ims_8_with_scalebars[i],
        plugin=None,
        check_contrast=True,
    )

___

# Computing environment

In [ ]:
%load_ext watermark

%watermark -v -p glob,os,re,numpy,pandas,skimage,bokeh,bokeh_catplot,jupyterlab